In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [2]:
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0",
            'Accept-Language': 'en;q=0.5'}

In [3]:
listaURL = []
pagina = 0
cantidad = 1

while True:   
    
    url = 'https://listado.mercadolibre.com.ar/computacion/monitores-accesorios/monitores/nuevo/monitor-led_Desde_'+str(cantidad)+'_NoIndex_True'
    page = requests.get(url,headers=HEADERS)
    
    if page.status_code >= 400:
        break
    else:    
        pagina += 1
        soup = BeautifulSoup(page.content,'html.parser')    
    
        for x in soup.find_all('a',class_ = 'ui-search-item__group__element shops__items-group-details ui-search-link'):
            if 'click1' not in str(x['href']):
                listaURL.append(x['href'])
    
        cantidad += 50
        
        time.sleep(4)


cantidad = None

print('Páginas visitadas: ', pagina)

Páginas visitadas:  38


In [6]:
cabeceras = ['Marca','Modelo','Tamaño de la pantalla','Tipo de resolución','Tipo de pantalla','Frecuencia de actualización recomendada','Precio','Link']

df = pd.DataFrame(columns = ['Marca','Modelo','Tamaño de la pantalla','Tipo de resolución','Tipo de pantalla','Frecuencia de actualización','Precio','Link'])

HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
len(listaURL)

1824

In [ ]:
for articuloURL in listaURL:
    timeoutException = False
    specsContent = ['','','','','','',0,'']
    rows = []
    
    while True:            
        if len(rows)>0:
            break
        else:
            try:
                page = requests.get(articuloURL,headers=HEADERS,timeout=5)                
            except:
                timeoutException = True
                print('Request timeout. Going to the next URL...')
                break
            soup = BeautifulSoup(page.text,'html')
            rows = soup.find_all('tr',class_ = 'andes-table__row ui-vpp-striped-specs__row')
            time.sleep(3)
                      
    if page.status_code == 200 and not timeoutException:

        for x in rows:
            cabecera = str(x.find('th').text.strip())            
            if cabecera in cabeceras:
                dato = str(x.find('td').text.strip())
                if cabecera == 'Marca':
                    specsContent[0] = dato
                elif cabecera == 'Modelo':
                    specsContent[1] = dato
                elif cabecera == 'Tamaño de la pantalla':
                    specsContent[2] = dato
                elif cabecera == 'Tipo de resolución':
                    specsContent[3] = dato
                elif cabecera == 'Tipo de pantalla':
                    specsContent[4] = dato
                elif 'Frecuencia de actualización' in cabecera:
                    specsContent[5] = dato

        try:          
            specsContent[6]=int(str(soup.find('span',class_='andes-money-amount__fraction').text.strip()).replace('.',''))
        except:
            specsContent[6]= -1
        
        specsContent[7] = str(articuloURL)
        
        df.loc[df.shape[0]] = specsContent

In [ ]:
print('Cantidad de registros:',str(len(df)))

In [ ]:
df

In [ ]:
df.to_csv('Monitores LED y LCD - Mercado Libre Argentina - Agosto 2023.csv')